# **Introduction**

In this notebook we will explore and benchmark different tree models (that is Decision Trees, Random Forest and Gradient Boosting) using a (toy) binary classification as a playground.   
Credit: this notebook was inspired by and built upon [this notebook](https://github.com/yandexdataschool/mlhep2019/blob/master/notebooks/day-2/02_decision_trees_and_ensembles.ipynb) from [MLHEP 2019 school](https://indico.cern.ch/event/768915/).

# **Decision Tree**

In [ ]:
# but firstly, do you remember how decision tree is constructed?

### **generate data**

Let's firstly generate some toy dataset with 2 features (this would make our studies very easy to visualize) and 2 classes:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

X_toy, y_toy = make_blobs(n_samples=400,
                          centers=[[0., 1.], [1., 2.]],
                          random_state=42)

plt.scatter(X_toy[:, 0], X_toy[:, 1], c=y_toy, alpha=0.8, cmap='bwr')
plt.xlabel('X1'), plt.ylabel('X2');

In [ ]:
from sklearn.metrics import accuracy_score
def plot_decision_boundary(clf,
                          X: np.ndarray,
                          y: np.ndarray,
                          grid_step: float=0.02,
                          cmap='bwr',
                          alpha:float=0.6,
                          axes=None
        ):
    """
    Plot the decision boundary of a classifier, visualize selected points
    Args:
      clf: a fitted model, must support predict method
      X[n_examples, n_features]: points where to evaluate the classifier
      y[n_examples]: true labels
      grid_step: decision boundary plottting grid
      alpha: opacity of the decision boundary
      axes(matplotlib.axes._subplots.AxesSubplot): axes where plot, if None, a new figure is created
    """
    
    # Define the grid
    x_top_left = X.min(axis=0) - 1
    x_bottom_right = X.max(axis=0) + 1
    grid_x0, grid_x1 = np.meshgrid(
         np.arange(x_top_left[0], x_bottom_right[0], grid_step),
         np.arange(x_top_left[1], x_bottom_right[1], grid_step)
      )
    
    # Calculate predictions on the grid
    y_pred_grid = clf.predict(
                        np.stack(
                              [
                                grid_x0.ravel(),
                                grid_x1.ravel()
                              ],
                              axis=1
                            )
                      ).reshape(grid_x1.shape)
    
    # Find optimal contour levels and make a filled
    # contour plot of predictions
    labels = np.sort(np.unique(y))
    labels = np.concatenate([[labels[0] - 1],
                             labels,
                             [labels[-1] + 1]])
    medians = (labels[1:] + labels[:-1]) / 2
    if axes is None:
      _, axes = plt.subplots()
    axes.contourf(grid_x0, grid_x1, y_pred_grid, cmap=cmap, alpha=alpha,
                 levels=medians)
    
    # Scatter data points on top of the plot,
    # with different styles for correct and wrong
    # predictions
    y_pred = clf.predict(X)
    axes.scatter(*X[y_pred==y].T, c=y[y_pred==y],
                marker='o', cmap=cmap, s=10, label='correct')
    axes.scatter(*X[y_pred!=y].T, c=y[y_pred!=y],
                marker='x', cmap=cmap, s=50, label='errors')

    # Dummy plot call to print the accuracy in the legend.
    axes.plot([], [], ' ',
             label='Accuracy = {:.3f}'.format(accuracy_score(y, y_pred)))
    axes.legend(loc='best')

### **decision boundary**

As we start from decision trees, let's import corresponding `DecisionTreeClassifier` class from `sklearn` library and fit it to the data.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=3) 
clf.fit(X_toy, y_toy)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
plot_decision_boundary(clf, X_toy, y_toy, axes=ax)

Trees themselves also can be visualized:

In [ ]:
from sklearn.tree import plot_tree

fig, ax = plt.subplots(figsize=(15, 15))
plot_tree(clf, ax=ax);

Once the training is done, we can predict class labels with a good old `predict()` method:

In [ ]:
clf.predict(X_toy)

or use `predict_proba()` to return probabilities for each class:

In [ ]:
# NB: strictly speaking, this output cannot be interpreted as probability of a class
# a calibration procedure is supposed to eliminate this difference, so please have a look here for some methods:
#  https://scikit-learn.org/stable/modules/calibration.html

clf.predict_proba(X_toy) 

### **Exercise 1**

Now firstl, let's investigate how the decision boundary depends on the tree depth. Maximum tree depth is defined by the `max_depth` parameter.   
Try out the following values: ``[1, 2, 3, 5, 10, 100]`` and make decision boundary plots for both train and test datasets (separately).

In [ ]:
from sklearn.model_selection import train_test_split

X_toy_train, X_toy_test, y_toy_train, y_toy_test = \
    train_test_split(X_toy, y_toy, test_size=0.25, random_state=42)

In [ ]:
depth_values = [1, 2, 3, 5, 10, 100]

fig, axes_matrix = plt.subplots(nrows=len(depth_values), ncols=2,
                                figsize=(2*5, 5*len(depth_values)))
for depth, (axes_train, axes_test) in zip(depth_values, axes_matrix):
  ### your code here ###
    pass

### **Exercise 2**

Previously, we were tuning only `max_depth` parameter, but as you might've remembered from the lecture, there's more of them. For example:

* `min_samples_split` – there should be at least this many samples to split further (default: 2)
* `min_samples_leaf` – there should be at least this many samples on one side of a split to consider it valid (default: 1)
* `max_leaf_nodes` – there can be at most this number of leaves in the tree (default: unlimited)
* `criterion` – the function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain (default: 'gini')
* `min_impurity_decrease` – node will be split if the split induces a decrease of the impurity greater than or equal to this value (default: 0)

Note: you can always look them up in the [DecisionTree API](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) or using `Tab` inside of the class instance declaration.

Now try to adjust these parameters to get the largest accuracy on the test set. How far can you reach?

In [ ]:
# tune me
tree_params = {'max_depth': None,
               'min_samples_split': 2,
               'min_samples_leaf': 1, 
               'max_leaf_nodes': None, 
               'criterion': 'gini',
               'min_impurity_decrease': 0,
              }

In [ ]:
clf = DecisionTreeClassifier(**tree_params)
clf.fit(X_toy_train, y_toy_train)

In [ ]:
fig, axes_matrix = plt.subplots(nrows=1, ncols=2, figsize=(2*5, 5))
plot_decision_boundary(clf, X_toy_train, y_toy_train, axes=axes_matrix[0])
plot_decision_boundary(clf, X_toy_test, y_toy_test, axes=axes_matrix[1])

# **Random Forest**

OK, once we got familiar with single decision trees, let's combine them together into a (presumably) more powerful Random Forest. Then we fit them to the same toy binary classification data and compare with the single tree results from above.

In [ ]:
# but firstly, do you remember how Random Forest is constructed?

### **Exercise 3**

Below we define a dictionary with RF parameters set to default values. Play around them and try to understand how each parameter affects the final result. Do you achieve higher performance with the ensemble comparing to a single decision tree? 

In [ ]:
# tune me
rf_params = {
    
    # tree params
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1, 
    'max_leaf_nodes': None, 
    'criterion': 'gini',
    'min_impurity_decrease': 0,
    
    # ensemble params
    'max_features': 'auto', # number of features to consider when looking for the best split. 'auto' = sqrt(n_features) 
    'n_estimators': 100,  # number of trees in ensemble
    'bootstrap': True, # whether to train each tree on a bootstrapped sample
    'oob_score': False, # whether to calculate out-of-bag score
    'n_jobs': -1 # number of parallel training processes, -1 means to use all available
}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(**rf_params) 
rf.fit(X_toy_train, y_toy_train)

In [ ]:
fig, axes_matrix = plt.subplots(nrows=1, ncols=2, figsize=(2*5, 5))
plot_decision_boundary(rf, X_toy_train, y_toy_train, axes=axes_matrix[0])
plot_decision_boundary(rf, X_toy_test, y_toy_test, axes=axes_matrix[1])

# **Gradient Boosting**

And finally, smart ensembling aka gradient boosting. Here for the sake of consistency we will use the one from `sklearn` library, but it is imperative to mention that there are more advanced libraries which can boost your trees. We will briefly describe them in a Bonus chapter below. 

In [ ]:
# but firstly, do you remember how Gradient Boosting is performed?

### **Exercise 4**

As before, try to change the parameters below and see how the result of prediction changes.

In [ ]:
# tune me
gbt_params = {
    
    # tree params
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1, 
    'max_leaf_nodes': None, 
    'min_impurity_decrease': 0,
    
    # ensemble params
    'loss': 'deviance', # loss function to be optimized; 'deviance' refers to logistic regression for classification with probabilistic outputs
    'learning_rate': 0.1, # learning rate shrinks the contribution of each tree by `learning_rate`
    'criterion': 'friedman_mse', # function to measure the quality of a split
    'max_features': 'auto', # number of features to consider when looking for the best split. 'auto' = sqrt(n_features) 
    'n_estimators': 100,  # number of trees in ensemble
    'subsample': 1.0, # fraction of samples to be used for fitting the individual base 
}

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbt = GradientBoostingClassifier(**gbt_params) 
gbt.fit(X_toy_train, y_toy_train)

In [ ]:
fig, axes_matrix = plt.subplots(nrows=1, ncols=2, figsize=(2*5, 5))
plot_decision_boundary(gbt, X_toy_train, y_toy_train, axes=axes_matrix[0])
plot_decision_boundary(gbt, X_toy_test, y_toy_test, axes=axes_matrix[1])

In [ ]:
# can you conclude which algorithm (Decision Tree, Random Forest, Gradient Boosting) performs better on this toy problem?

# **Bonus**

Finally, as we mentioned earlier, there are several libraries available on the market which can perform boosting on trees. It is highly likely that you will use them in the future, so it's important to mentioned them here. It should also be emphasized that there is no "best" library - they are all "best" in their own way. So the choice depends largely on type of the problem you want to solve and on your tastes/preferences. Currently, these are the **most prominent libraries for boosting** out there, and we encourage you to read their description and documentations to get the general understanding of each advantages:

* [**XGBoost**](https://xgboost.readthedocs.io/en/latest/)
* [**LightGBM**](https://lightgbm.readthedocs.io/en/latest/)
* [**CatBoost**](https://catboost.ai/)

Here we will do **a very simple and dummy benchmarking** of them - no hyperparameter tuning, just fitting models in a regression setting with their libraries' default parameters using some toy data. However, we highly encourage you to do the same on a more complicated dataset and with a more elaborated hyperparameters tuning. For example, [this data](https://www.kaggle.com/c/higgs-boson/data) from Higgs Kaggle challenge is a good place to start, or also it would be interesting to try them out in homework of this module.

### **generate data**

In [ ]:
np.random.seed(42) # to reproduce the same random state
train_size = 20 # number of points to train on

x_train = np.sort(np.random.uniform(-2., 2., size=train_size))
y_train = x_train**3 - 1.5 * x_train**2 - 2.*x_train - 1
y_train += np.random.normal(scale=0.7, size=train_size) # adding some noize to the data
x_train = x_train.reshape(-1, 1) 

In [ ]:
np.random.seed(42)
test_size = 50 

# also generate test data
x_test = np.sort(np.random.uniform(-2., 2., size=test_size))
y_test = x_test**3 - 1.5 * x_test**2 - 2.*x_test - 1
y_test += np.random.normal(scale=0.7, size=test_size) 
x_test = x_test.reshape(-1, 1) 

In [ ]:
plt.scatter(x_train, y_train, s=30, color='gray', label='train')
plt.scatter(x_test, y_test, s=30, color='black', label='test')
plt.grid()
plt.legend()
plt.show()

In [ ]:
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb

In [ ]:
def plot_predictions(x_values, y_true_values, y_pred_values, ax, label, color=None):
  """
  Function that plots the data points and
  model prediction
  """
  ax.plot(x_values, y_true_values, 'o', label='true', markersize=6, color='gray')
  ax.plot(x_values, y_pred_values, '-', label=label, color=color)

In [ ]:
regressor_lgb = lgb.LGBMRegressor(n_estimators=5)
regressor_xgb = xgb.XGBRegressor(n_estimators=5)
regressor_ctb = ctb.CatBoostRegressor(n_estimators=5)

In [ ]:
regressor_lgb.fit(x_train, y_train)
regressor_xgb.fit(x_train, y_train)
regressor_ctb.fit(x_train, y_train)

In [ ]:
y_train_preds_lgb = regressor_lgb.predict(x_train)
y_train_preds_xgb = regressor_xgb.predict(x_train)
y_train_preds_ctb = regressor_ctb.predict(x_train)

y_test_preds_lgb = regressor_lgb.predict(x_test)
y_test_preds_xgb = regressor_xgb.predict(x_test)
y_test_preds_ctb = regressor_ctb.predict(x_test)

In [ ]:
fig, axes_matrix = plt.subplots(nrows=1, ncols=2,
                                figsize=(2*5, 5))


plot_predictions(x_train, y_train, y_train_preds_lgb, label='LightGBM', ax=axes_matrix[0])
plot_predictions(x_train, y_train, y_train_preds_xgb, label='XGBoost', ax=axes_matrix[0])
plot_predictions(x_train, y_train, y_train_preds_ctb, label='CatBoost', ax=axes_matrix[0])
axes_matrix[0].legend()
axes_matrix[0].grid()
axes_matrix[0].set_title('train')

plot_predictions(x_test, y_test, y_test_preds_lgb, label='LightGBM', ax=axes_matrix[1])
plot_predictions(x_test, y_test, y_test_preds_xgb, label='XGBoost', ax=axes_matrix[1])
plot_predictions(x_test, y_test, y_test_preds_ctb, label='CatBoost', ax=axes_matrix[1])
axes_matrix[1].legend()
axes_matrix[1].grid()
axes_matrix[1].set_title('test')